In [1]:
import numpy as np
import pandas as pd
import folium
import branca.colormap as cm
import geopandas as gpd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon

# Import data and convert to geopandas df

In [2]:
trips = pd.read_csv('../data/processed/trips.csv')

In [3]:
trips = trips[(trips['start_lng'] != trips['end_lng']) | (trips['start_lat'] != trips['end_lat'])]

In [4]:
geo_trips_start = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.start_lng, trips.start_lat))

In [6]:
geo_trips_start.head(2)

,bike,bike_type,identification,start_time,end_time,duration_sec,start_lng,start_lat,end_lng,end_lat,start_place,end_place,geometry
8,20507,29,14776184,2019-04-21 17:34:00,2019-04-21 17:47:00,780.0,8.884911,53.078923,8.899906,53.078711,2985.0,0.0,POINT (8.88491 53.07892)
45,20507,29,16155376,2019-05-23 15:33:00,2019-05-23 15:40:00,420.0,8.884911,53.078923,8.876828,53.078004,2985.0,0.0,POINT (8.88491 53.07892)


In [7]:
geo_trips_end = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.end_lng, trips.end_lat))
geo_trips_end.head(2)

,bike,bike_type,identification,start_time,end_time,duration_sec,start_lng,start_lat,end_lng,end_lat,start_place,end_place,geometry
8,20507,29,14776184,2019-04-21 17:34:00,2019-04-21 17:47:00,780.0,8.884911,53.078923,8.899906,53.078711,2985.0,0.0,POINT (8.89991 53.07871)
45,20507,29,16155376,2019-05-23 15:33:00,2019-05-23 15:40:00,420.0,8.884911,53.078923,8.876828,53.078004,2985.0,0.0,POINT (8.87683 53.07800)


# Import PLZ Deutschland - Bremen data set
https://public.opendatasoft.com/explore/dataset/postleitzahlen-deutschland/table/?refine.note=Bremen

In [8]:
plz_df = gpd.read_file('../data/external/plz_bremen.geojson')

In [9]:
plz_df.head(2)

,note,plz,geometry
0,Bremen,28199,"POLYGON ((8.76410 53.06703, 8.77230 53.06924, ..."
1,Bremen,28309,"POLYGON ((8.86527 53.03539, 8.86535 53.03602, ..."


## Function to get center of geo data set

In [10]:
def get_center(gpd_df):

    x_map = gpd_df.centroid.x.mean()
    y_map = gpd_df.centroid.y.mean()
    
    return [y_map, x_map]

## Visualization of plz areas of Bremen

Note: 28329 comprises two disjunct areas - anyway the task is to count starts and end per PLZ

In [11]:
plz_bremen = folium.Map(location=get_center(plz_df), zoom_start=11)

folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(plz_bremen)

folium.Choropleth(geo_data=plz_df).add_to(plz_bremen)

folium.LayerControl().add_to(plz_bremen)

plz_bremen

# Create new data frame with plz, the corresponding geometry and number of trips started and ended in this area

## Starts

In [40]:
geo_trips_start['count'] = 1

dfsjoin = gpd.sjoin(plz_df,geo_trips_start[['geometry','count']])

dfpivot = pd.pivot_table(dfsjoin,index='plz',columns='count',aggfunc={'count':len})

dfpivot.columns = dfpivot.rename(columns={1:'trips_count'}).columns.droplevel()

plz_starts = plz_df.merge(dfpivot, how='left',on='plz')

/Users/timockenga/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [41]:
plz_starts

,note,plz,geometry,trips_count
0,Bremen,28199,"POLYGON ((8.76410 53.06703, 8.77230 53.06924, ...",14887
1,Bremen,28309,"POLYGON ((8.86527 53.03539, 8.86535 53.03602, ...",677
2,Bremen,28717,"POLYGON ((8.66585 53.17528, 8.66609 53.17565, ...",5
3,Bremen,28755,"POLYGON ((8.58081 53.17585, 8.58391 53.17748, ...",12
4,Bremen,28217,"POLYGON ((8.74623 53.10582, 8.75125 53.10755, ...",9568
5,Bremen,28325,"POLYGON ((8.93050 53.05900, 8.93386 53.06806, ...",56
6,Bremen,28209,"POLYGON ((8.81358 53.09035, 8.81453 53.09080, ...",7226
7,Bremen,28219,"POLYGON ((8.76607 53.10855, 8.76712 53.10949, ...",956
8,Bremen,28197,"POLYGON ((8.64461 53.12789, 8.64713 53.12853, ...",1560
9,Bremen,28277,"POLYGON ((8.79180 53.03632, 8.79368 53.03810, ...",812


In [42]:
# Can be used for checking individual quantiles.
myscale = (plz_starts['trips_count'].quantile((0,0.6,0.75,0.9,0.98,1))).tolist()

m = folium.Map(location=get_center(plz_starts), zoom_start=11)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

folium.Choropleth(
    geo_data=plz_starts,
    name='choropleth',
    data=plz_starts,
    columns=['plz','trips_count'],
    key_on="feature.properties.plz",
    fill_color='YlGnBu',
    #threshold_scale=myscale,
    fill_opacity=0.5,
    line_opacity=0.5,
    legend_name='Number of trips'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [43]:
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
popup = folium.features.GeoJson(
    plz_starts,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['plz','trips_count'],
        aliases=['Postleitzahl: ','Number of trips in 2019: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

m.add_child(popup)
m.keep_in_front(popup)
folium.LayerControl().add_to(m)
m

In [44]:
colormap = cm.linear.YlGnBu_09.scale(0, plz_starts.trips_count.max())

mymap = folium.Map(location=get_center(plz_starts), zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
colormap.caption = "number of starts in 2019"
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['trips_count']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
trips_count=folium.features.GeoJson(
        plz_starts,
        style_function=style_function,
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=['plz','trips_count'],
            aliases=['Postleitzahl: ','Number of trips in 2019: '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
colormap.add_to(mymap)
mymap.add_child(trips_count)
mymap

## Ends

In [45]:
geo_trips_end['count'] = 1

dfsjoin = gpd.sjoin(plz_df,geo_trips_end[['geometry','count']])

dfpivot = pd.pivot_table(dfsjoin,index='plz',columns='count',aggfunc={'count':len})

dfpivot.columns = dfpivot.rename(columns={1:'trips_count'}).columns.droplevel()

plz_ends = plz_df.merge(dfpivot, how='left',on='plz')

/Users/timockenga/anaconda3/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!({'init': 'epsg:4326'} != None)
  "(%s != %s)" % (left_df.crs, right_df.crs)


In [46]:
plz_ends

,note,plz,geometry,trips_count
0,Bremen,28199,"POLYGON ((8.76410 53.06703, 8.77230 53.06924, ...",14887
1,Bremen,28309,"POLYGON ((8.86527 53.03539, 8.86535 53.03602, ...",677
2,Bremen,28717,"POLYGON ((8.66585 53.17528, 8.66609 53.17565, ...",5
3,Bremen,28755,"POLYGON ((8.58081 53.17585, 8.58391 53.17748, ...",12
4,Bremen,28217,"POLYGON ((8.74623 53.10582, 8.75125 53.10755, ...",9568
5,Bremen,28325,"POLYGON ((8.93050 53.05900, 8.93386 53.06806, ...",56
6,Bremen,28209,"POLYGON ((8.81358 53.09035, 8.81453 53.09080, ...",7226
7,Bremen,28219,"POLYGON ((8.76607 53.10855, 8.76712 53.10949, ...",956
8,Bremen,28197,"POLYGON ((8.64461 53.12789, 8.64713 53.12853, ...",1560
9,Bremen,28277,"POLYGON ((8.79180 53.03632, 8.79368 53.03810, ...",812


In [47]:
# Can be used for checking individual quantiles.
myscale = (plz_ends['trips_count'].quantile((0,0.6,0.75,0.9,0.98,1))).tolist()

m = folium.Map(location=get_center(plz_ends), zoom_start=11)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(m)

folium.Choropleth(
    geo_data=plz_ends,
    name='choropleth',
    data=plz_ends,
    columns=['plz','trips_count'],
    key_on="feature.properties.plz",
    fill_color='YlGnBu',
    #threshold_scale=myscale,
    fill_opacity=0.5,
    line_opacity=0.5,
    legend_name='Number of trips'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [28]:
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
popup = folium.features.GeoJson(
    dfpolynew,
    style_function=style_function, 
    control=False,
    highlight_function=highlight_function, 
    tooltip=folium.features.GeoJsonTooltip(
        fields=['plz','trips_count'],
        aliases=['Postleitzahl: ','Number of trips in 2019: '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)

m.add_child(popup)
m.keep_in_front(popup)
folium.LayerControl().add_to(m)
m

In [48]:
colormap = cm.linear.YlGnBu_09.scale(0, plz_ends.trips_count.max())

mymap = folium.Map(location=get_center(plz_ends), zoom_start=11,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)
colormap.caption = "number of starts in 2019"
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':colormap(x['properties']['trips_count']), 
                            'fillOpacity':0.75}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1}
trips_count=folium.features.GeoJson(
        plz_ends,
        style_function=style_function,
        control=False,
        highlight_function=highlight_function,
        tooltip=folium.features.GeoJsonTooltip(fields=['plz','trips_count'],
            aliases=['Postleitzahl: ','Number of trips in 2019: '],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
            sticky=True
        )
    )
colormap.add_to(mymap)
mymap.add_child(trips_count)
mymap

In [50]:
pd.merge(plz_starts,plz_ends,on='plz')[['trips_count_x','trips_count_y']]

,trips_count_x,trips_count_y
0,14887,14887
1,677,677
2,5,5
3,12,12
4,12,12
5,12,12
6,12,12
7,9568,9568
8,56,56
9,7226,7226
